In [2]:
!git clone "https://github.com/smartyfh/MultiWOZ2.4.git"

fatal: destination path 'MultiWOZ2.4' already exists and is not an empty directory.


In [9]:
%cd "MultiWOZ2.4"
!python create_data.py
%cd ".."

c:\Users\Xiaocheng Yang\Documents\Code Files\Respact\multiwoz\MultiWOZ2.4
Create WOZ-like dialogues. Get yourself a coffee, this might take a while.
Divide dialogues...
# of dialogues: Train 8420, Val 1000, Test 999
c:\Users\Xiaocheng Yang\Documents\Code Files\Respact\multiwoz


In [ ]:
from prompt import eval_prompt_respact_v5_0_shot
from evaluator import evaluator
from openai import OpenAI, AzureOpenAI
from getpass import getpass
import json
import random

with open("MultiWOZ2.4/data/mwz2.4/test_dials.json", "r") as f:
    data = json.load(f)

with open("MultiWOZ2.4/data/mwz24/data.json", "r") as f:
    old_data = json.load(f)

# Data Process

In [ ]:
sample = 100 # Interger or None

In [12]:
domains_in_interest = ["taxi", "restaurant", "attraction", "hotel", "train"]

In [13]:
def get_domains_samples(data, domains_in_interest):
    domains_samples = {domain: list() for domain in domains_in_interest}
    processed_data = dict()
    for dialogue_content in data:
        for domain in dialogue_content["domains"]:
            if domain in domains_in_interest:
                domains_samples[domain].append(dialogue_content["dialogue_idx"])
                processed_data[dialogue_content["dialogue_idx"]] = dialogue_content
    return domains_samples, processed_data

In [14]:
domains_samples, data = get_domains_samples(data, domains_in_interest)

In [ ]:
total_samples = set()
total_length = 0

for domain, samples in domains_samples.items():
    total_samples = total_samples.union(set(samples))
    total_length += len(samples)
if sample:
    total_samples = sorted(list(total_samples))
    random.seed(10)
    total_samples = random.sample(total_samples, sample)
else:
    total_samples = list(total_samples)

print(len(total_samples))
print(total_samples[:5])
print(total_length)

1
['PMUL3107.json']
1915


# Client Config

In [ ]:
client_config = {
    "client": AzureOpenAI(
        azure_endpoint = "azure_endpoint",
        api_key = getpass("🔑 Enter your OpenAI API key: "),
        api_version="api_version"
    ),
    "model": "gpt-4o-mini",
}

# client_config = {
#     "client": OpenAI(
#         base_url = "base_url",
#         api_key = getpass("🔑 Enter your OpenAI API key: "),
#     ),
#     "model": "gpt-4o-mini",
# }

In [ ]:
user_client_config = {
    "client": AzureOpenAI(
        azure_endpoint = "azure_endpoint",
        api_key = getpass("🔑 Enter your OpenAI API key: "),
        api_version="api_version"
    ),
    "model": "gpt-4o-mini",
}

# user_client_config = {
#     "client": OpenAI(
#         base_url = "base_url",
#         api_key = getpass("🔑 Enter your OpenAI API key: "),
#     ),
#     "model": "gpt-4o-mini",
# }

# Evaluation

In [ ]:
online_evaluator = evaluator(
    data,
    old_data,
    total_samples,
    prompt=eval_prompt_respact_v5_0_shot,
    client_config=client_config,
    user_client_config=user_client_config,
    online=True
)

In [9]:
save_dir = "eval_result_online-41.json"

online_evaluator.evaluate()
online_evaluator.save_result(save_dir)

100%|██████████| 100/100 [1:26:23<00:00, 51.83s/it]
